In [ ]:
# Prétraitement Multi-Années (2018-2023)

**Objectif** : Calculer ETb et WPb pour toutes les années disponibles

## Contenu
1. Charger le masque cropland (créé dans notebook 02)
2. Boucle sur toutes les années (2018-2023)
3. Calculer ETb et WPb pour chaque année
4. Créer un DataFrame temporel avec statistiques
5. Visualisations temporelles
6. Sauvegarder les résultats

**Rappel** :
- **ETb** = Eau d'irrigation utilisée (AETI - Pluie_utile)
- **WPb** = Efficacité de production (TBP / ETb)

In [ ]:
print("✅ Notebook 03 - Prétraitement multi-années terminé\n")
print("📁 Fichiers créés:")
print(f"  - {len(etb_rasters)} rasters ETb annuels")
print(f"  - {len(wpb_rasters)} rasters WPb annuels")
print(f"  - {stats_output}")
print(f"  - results/graphs/etb_temporal_analysis.png")
if wpb_rasters:
    print(f"  - results/graphs/wpb_temporal_analysis.png")
print(f"  - results/maps/etb_multiyear_maps.png")
if wpb_rasters:
    print(f"  - results/maps/wpb_multiyear_maps.png")

print("\n📊 Résumé:")
print(f"  - Période analysée: {min(years)} - {max(years)}")
print(f"  - ETb moyen (sur toutes les années): {df_etb['etb_mean'].mean():.1f} mm/an")
if wpb_stats_list:
    print(f"  - WPb moyen (sur toutes les années): {df_wpb['wpb_mean'].mean():.2f} kg/m³")

print("\n➡️ Prochaines étapes (Notebook 04):")
print("  1. Télécharger le shapefile des gouvernorats de Tunisie")
print("  2. Calculer les statistiques zonales (ETb et WPb par gouvernorat)")
print("  3. Créer des cartes choroplèthes par gouvernorat")
print("  4. Identifier les gouvernorats les plus/moins performants")

print("\n💡 Données manquantes pour AWP:")
print("  - GVA_a: Valeur ajoutée brute agricole (USD)")
print("  - V_a: Volume d'eau retirés pour l'agriculture (m³)")
print("  - c_r: Proportion rainfed vs irrigué")
print("  Source: FAO AQUASTAT, World Bank, INS Tunisia")

## 11. Résumé et Prochaines Étapes

In [ ]:
if wpb_rasters and len(wpb_rasters) > 0:
    # Créer grille de sous-graphiques
    n_years = len(wpb_rasters)
    n_cols = 3
    n_rows = (n_years + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 6 * n_rows))
    axes = axes.flatten() if n_years > 1 else [axes]
    
    # Colormap
    colors = ['#8c510a', '#d8b365', '#f6e8c3', '#c7eae5', '#5ab4ac', '#01665e']
    cmap = LinearSegmentedColormap.from_list('productivity', colors)
    
    # Calculer vmin/vmax global
    all_wpb = np.concatenate([wpb_rasters[y].flatten() for y in sorted(wpb_rasters.keys())])
    vmin, vmax = np.nanpercentile(all_wpb, [2, 98])
    
    # Tracer chaque année
    for idx, year in enumerate(sorted(wpb_rasters.keys())):
        ax = axes[idx]
        
        im = ax.imshow(wpb_rasters[year], cmap=cmap, vmin=vmin, vmax=vmax, interpolation='nearest')
        ax.set_title(f'WPb {year}\nMoyenne: {df_wpb[df_wpb["year"]==year]["wpb_mean"].values[0]:.2f} kg/m³', 
                     fontsize=12, fontweight='bold')
        ax.axis('off')
        
        # Colorbar individuelle
        cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        cbar.set_label('WPb (kg/m³)', fontsize=10)
    
    # Cacher les axes vides
    for idx in range(n_years, len(axes)):
        axes[idx].axis('off')
    
    plt.suptitle('Productivité de la Biomasse - Tunisie (2018-2023)', 
                 fontsize=16, fontweight='bold', y=0.998)
    plt.tight_layout()
    plt.savefig(f"{processed_dir}/../results/maps/wpb_multiyear_maps.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    print("💾 Cartes sauvegardées: results/maps/wpb_multiyear_maps.png")
else:
    print("⚠️ Pas de données WPb disponibles pour les cartes")

## 10. Cartes Multi-Années - WPb

Visualiser la productivité de la biomasse pour chaque année

In [ ]:
# Créer grille de sous-graphiques
n_years = len(etb_rasters)
n_cols = 3
n_rows = (n_years + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 6 * n_rows))
axes = axes.flatten() if n_years > 1 else [axes]

# Colormap
colors = ['#d7191c', '#fdae61', '#ffffbf', '#abd9e9', '#2c7bb6']
cmap = LinearSegmentedColormap.from_list('water', colors[::-1])

# Calculer vmin/vmax global
all_etb = np.concatenate([etb_rasters[y].flatten() for y in sorted(etb_rasters.keys())])
vmin, vmax = np.nanpercentile(all_etb, [2, 98])

# Tracer chaque année
for idx, year in enumerate(sorted(etb_rasters.keys())):
    ax = axes[idx]
    
    im = ax.imshow(etb_rasters[year], cmap=cmap, vmin=vmin, vmax=vmax, interpolation='nearest')
    ax.set_title(f'ETb {year}\nMoyenne: {df_etb[df_etb["year"]==year]["etb_mean"].values[0]:.1f} mm/an', 
                 fontsize=12, fontweight='bold')
    ax.axis('off')
    
    # Colorbar individuelle
    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label('ETb (mm/an)', fontsize=10)

# Cacher les axes vides
for idx in range(n_years, len(axes)):
    axes[idx].axis('off')

plt.suptitle('Évapotranspiration Bleue - Tunisie (2018-2023)', 
             fontsize=16, fontweight='bold', y=0.998)
plt.tight_layout()
plt.savefig(f"{processed_dir}/../results/maps/etb_multiyear_maps.png", dpi=300, bbox_inches='tight')
plt.show()

print("💾 Cartes sauvegardées: results/maps/etb_multiyear_maps.png")

## 9. Cartes Multi-Années - ETb

Visualiser la distribution spatiale pour chaque année

In [ ]:
if wpb_stats_list and len(wpb_stats_list) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # WPb moyen
    ax = axes[0, 0]
    ax.plot(df_wpb['year'], df_wpb['wpb_mean'], marker='o', linewidth=2, 
            markersize=8, color='#1a9850')
    ax.fill_between(df_wpb['year'], 
                     df_wpb['wpb_mean'] - df_wpb['wpb_std'], 
                     df_wpb['wpb_mean'] + df_wpb['wpb_std'], 
                     alpha=0.3, color='#1a9850')
    ax.set_xlabel('Année', fontsize=12, fontweight='bold')
    ax.set_ylabel('WPb (kg/m³)', fontsize=12, fontweight='bold')
    ax.set_title('WPb Moyen (± écart-type)', fontsize=13, fontweight='bold')
    ax.grid(True, alpha=0.3)
    
    # TBP moyen
    ax = axes[0, 1]
    ax.plot(df_wpb['year'], df_wpb['tbp_mean'], marker='s', linewidth=2, 
            markersize=8, color='#762a83')
    ax.set_xlabel('Année', fontsize=12, fontweight='bold')
    ax.set_ylabel('TBP (kg/ha)', fontsize=12, fontweight='bold')
    ax.set_title('Production de Biomasse Moyenne', fontsize=13, fontweight='bold')
    ax.grid(True, alpha=0.3)
    
    # WPb min/max
    ax = axes[1, 0]
    ax.fill_between(df_wpb['year'], df_wpb['wpb_min'], df_wpb['wpb_max'], 
                    alpha=0.3, color='#a6dba0', label='Plage [min-max]')
    ax.plot(df_wpb['year'], df_wpb['wpb_median'], marker='o', linewidth=2, 
            color='#1b7837', label='Médiane')
    ax.set_xlabel('Année', fontsize=12, fontweight='bold')
    ax.set_ylabel('WPb (kg/m³)', fontsize=12, fontweight='bold')
    ax.set_title('Plage de Variation WPb', fontsize=13, fontweight='bold')
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)
    
    # Tableau récapitulatif
    ax = axes[1, 1]
    ax.axis('off')
    
    table_data = []
    for _, row in df_wpb.iterrows():
        table_data.append([
            int(row['year']),
            f"{row['wpb_mean']:.2f}",
            f"{row['wpb_median']:.2f}",
            f"{row['tbp_mean']:.0f}"
        ])
    
    table = ax.table(cellText=table_data,
                    colLabels=['Année', 'WPb moy.', 'WPb méd.', 'TBP moy.'],
                    cellLoc='center',
                    loc='center',
                    colWidths=[0.2, 0.25, 0.25, 0.25])
    
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2)
    
    # Style du header
    for i in range(4):
        table[(0, i)].set_facecolor('#1a9850')
        table[(0, i)].set_text_props(weight='bold', color='white')
    
    ax.set_title('Résumé WPb (kg/m³) et TBP (kg/ha)', fontsize=13, fontweight='bold', pad=20)
    
    plt.suptitle('Évolution Temporelle - Productivité de la Biomasse (WPb)', 
                 fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.savefig(f"{processed_dir}/../results/graphs/wpb_temporal_analysis.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    print("💾 Graphique sauvegardé: results/graphs/wpb_temporal_analysis.png")
else:
    print("⚠️ Pas de données WPb disponibles pour la visualisation")

## 8. Visualisation Temporelle - WPb

Évolution de la productivité de la biomasse dans le temps

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# ETb moyen
ax = axes[0, 0]
ax.plot(df_etb['year'], df_etb['etb_mean'], marker='o', linewidth=2, markersize=8, color='#2c7bb6')
ax.fill_between(df_etb['year'], 
                 df_etb['etb_mean'] - df_etb['etb_std'], 
                 df_etb['etb_mean'] + df_etb['etb_std'], 
                 alpha=0.3, color='#2c7bb6')
ax.set_xlabel('Année', fontsize=12, fontweight='bold')
ax.set_ylabel('ETb (mm/an)', fontsize=12, fontweight='bold')
ax.set_title('ETb Moyen (± écart-type)', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3)

# AETI vs PCP
ax = axes[0, 1]
ax.plot(df_etb['year'], df_etb['aeti_mean'], marker='o', linewidth=2, label='AETI', color='#d7191c')
ax.plot(df_etb['year'], df_etb['pcp_mean'], marker='s', linewidth=2, label='PCP', color='#2c7bb6')
ax.set_xlabel('Année', fontsize=12, fontweight='bold')
ax.set_ylabel('mm/an', fontsize=12, fontweight='bold')
ax.set_title('AETI vs Précipitations', fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# ETb min/max
ax = axes[1, 0]
ax.fill_between(df_etb['year'], df_etb['etb_min'], df_etb['etb_max'], 
                alpha=0.3, color='#abdda4', label='Plage [min-max]')
ax.plot(df_etb['year'], df_etb['etb_median'], marker='o', linewidth=2, 
        color='#2b83ba', label='Médiane')
ax.set_xlabel('Année', fontsize=12, fontweight='bold')
ax.set_ylabel('ETb (mm/an)', fontsize=12, fontweight='bold')
ax.set_title('Plage de Variation ETb', fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Tableau récapitulatif
ax = axes[1, 1]
ax.axis('off')

table_data = []
for _, row in df_etb.iterrows():
    table_data.append([
        int(row['year']),
        f"{row['etb_mean']:.1f}",
        f"{row['etb_median']:.1f}",
        f"{row['aeti_mean']:.1f}",
        f"{row['pcp_mean']:.1f}"
    ])

table = ax.table(cellText=table_data,
                colLabels=['Année', 'ETb moy.', 'ETb méd.', 'AETI moy.', 'PCP moy.'],
                cellLoc='center',
                loc='center',
                colWidths=[0.15, 0.2, 0.2, 0.2, 0.2])

table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2)

# Style du header
for i in range(5):
    table[(0, i)].set_facecolor('#2c7bb6')
    table[(0, i)].set_text_props(weight='bold', color='white')

ax.set_title('Résumé des Statistiques (mm/an)', fontsize=13, fontweight='bold', pad=20)

plt.suptitle('Évolution Temporelle - Évapotranspiration Bleue (ETb)', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig(f"{processed_dir}/../results/graphs/etb_temporal_analysis.png", dpi=300, bbox_inches='tight')
plt.show()

print("💾 Graphique sauvegardé: results/graphs/etb_temporal_analysis.png")

## 7. Visualisation Temporelle - ETb

Évolution de l'évapotranspiration bleue dans le temps

In [ ]:
# Créer DataFrame ETb
df_etb = pd.DataFrame(etb_stats_list)
df_etb = df_etb.sort_values('year')

print("📊 Statistiques ETb (2018-2023):\n")
print(df_etb.to_string(index=False))

# Créer DataFrame WPb
if wpb_stats_list:
    df_wpb = pd.DataFrame(wpb_stats_list)
    df_wpb = df_wpb.sort_values('year')
    
    print("\n\n📊 Statistiques WPb (2018-2023):\n")
    print(df_wpb.to_string(index=False))
    
    # Fusionner les deux DataFrames
    df_combined = pd.merge(df_etb, df_wpb, on='year', how='left', suffixes=('_etb', '_wpb'))
else:
    df_combined = df_etb

# Sauvegarder
stats_output = f"{processed_dir}/temporal_statistics.csv"
df_combined.to_csv(stats_output, index=False)

print(f"\n\n💾 Statistiques sauvegardées: {stats_output}")

## 6. Créer DataFrame Temporel

Organiser les statistiques dans un tableau

In [ ]:
# Années à traiter
years = [2018, 2019, 2020, 2021, 2022, 2023]

# Dimensions de référence (du masque cropland)
reference_shape = cropland_mask.shape
reference_transform = mask_transform
reference_crs = mask_crs

# Stocker les résultats
etb_stats_list = []
wpb_stats_list = []
etb_rasters = {}
wpb_rasters = {}

# Créer répertoires de sortie
os.makedirs(f"{processed_dir}/ETb_annual", exist_ok=True)
os.makedirs(f"{processed_dir}/WPb_annual", exist_ok=True)

print("🔄 Traitement multi-années\n")
print(f"📅 Années: {years}")
print(f"📐 Dimensions de référence: {reference_shape}")
print(f"\n{'='*60}\n")

# Boucle sur les années
for year in years:
    try:
        # Calculer ETb
        etb_annual, etb_cropland, etb_stats = calculate_etb(
            year, cropland_mask, reference_transform, reference_crs, reference_shape
        )
        
        # Sauvegarder ETb
        etb_output = f"{processed_dir}/ETb_annual/ETb_{year}.tif"
        with rasterio.open(etb_output, 'w', **mask_profile) as dst:
            dst.write(etb_annual, 1)
        print(f"💾 ETb sauvegardé: {etb_output}")
        
        # Stocker
        etb_stats_list.append(etb_stats)
        etb_rasters[year] = etb_cropland
        
        # Calculer WPb
        wpb_annual, wpb_cropland, wpb_stats = calculate_wpb(
            year, etb_annual, cropland_mask, reference_transform, reference_crs, reference_shape
        )
        
        if wpb_stats is not None:
            # Sauvegarder WPb
            wpb_output = f"{processed_dir}/WPb_annual/WPb_{year}.tif"
            with rasterio.open(wpb_output, 'w', **mask_profile) as dst:
                dst.write(wpb_annual, 1)
            print(f"💾 WPb sauvegardé: {wpb_output}")
            
            # Stocker
            wpb_stats_list.append(wpb_stats)
            wpb_rasters[year] = wpb_cropland
        
    except Exception as e:
        print(f"❌ Erreur pour {year}: {e}")
        continue

print(f"\n{'='*60}")
print(f"✅ Traitement terminé")
print(f"  ETb calculé pour {len(etb_stats_list)} années")
print(f"  WPb calculé pour {len(wpb_stats_list)} années")
print(f"{'='*60}\n")

## 5. Traitement Multi-Années (2018-2023)

Boucle sur toutes les années disponibles

In [ ]:
def calculate_wpb(year, etb_annual, cropland_mask, reference_transform, reference_crs, reference_shape):
    """
    Calculer WPb pour une année donnée
    
    Parameters:
    -----------
    year : int
        Année à traiter
    etb_annual : np.ndarray
        ETb annuel (mm/an)
    cropland_mask : np.ndarray
        Masque des terres cultivées
    reference_transform : affine.Affine
        Transformation de référence
    reference_crs : CRS
        Système de coordonnées de référence
    reference_shape : tuple
        Dimensions (height, width) de référence
    
    Returns:
    --------
    wpb_annual : np.ndarray
        WPb annuel (kg/m³)
    wpb_cropland : np.ndarray
        WPb masqué sur les terres cultivées
    stats : dict
        Statistiques de WPb
    """
    
    # Charger TBP
    tbp_file = f"{data_dir}/TBP/TBP_L2_{year}.tif"
    
    if not os.path.exists(tbp_file):
        print(f"  ⚠️ TBP non disponible pour {year}")
        return None, None, None
    
    with rasterio.open(tbp_file) as src:
        tbp_raw = src.read(1).astype(np.float32)
        tbp_raw[tbp_raw < 0] = np.nan
        tbp_transform = src.transform
        tbp_crs = src.crs
    
    print(f"✓ TBP chargé: {tbp_raw.shape}")
    
    # Aligner TBP si nécessaire
    if tbp_raw.shape != reference_shape:
        print(f"  ⚠️ Rééchantillonnage TBP: {tbp_raw.shape} → {reference_shape}")
        tbp_annual = align_raster_to_reference(
            tbp_raw, tbp_transform, tbp_crs,
            reference_shape, reference_transform, reference_crs,
            resampling_method=Resampling.bilinear
        )
    else:
        tbp_annual = tbp_raw
    
    # Convertir ETb de mm/an en m³/ha
    # 1 mm/an = 10 m³/ha
    etb_m3_ha = etb_annual * 10
    
    # Calculer WPb = TBP / ETb (kg/m³)
    # Éviter division par zéro
    wpb_annual = np.where(etb_m3_ha > 0, tbp_annual / etb_m3_ha, np.nan)
    
    # Appliquer le masque cropland
    wpb_cropland = wpb_annual * cropland_mask
    wpb_cropland[cropland_mask == 0] = np.nan
    
    # Statistiques
    stats = {
        'year': year,
        'wpb_min': float(np.nanmin(wpb_cropland)),
        'wpb_max': float(np.nanmax(wpb_cropland)),
        'wpb_mean': float(np.nanmean(wpb_cropland)),
        'wpb_median': float(np.nanmedian(wpb_cropland)),
        'wpb_std': float(np.nanstd(wpb_cropland)),
        'tbp_mean': float(np.nanmean(tbp_annual))
    }
    
    print(f"\n📊 Statistiques WPb {year} (cropland):")
    print(f"  Moyenne: {stats['wpb_mean']:.2f} kg/m³")
    print(f"  Médiane: {stats['wpb_median']:.2f} kg/m³")
    print(f"  Min:     {stats['wpb_min']:.2f} kg/m³")
    print(f"  Max:     {stats['wpb_max']:.2f} kg/m³")
    
    return wpb_annual, wpb_cropland, stats

print("✓ Fonction calculate_wpb définie")

## 4. Fonction de Calcul WPb

Calculer la productivité de la biomasse (TBP / ETb)

In [ ]:
def calculate_etb(year, cropland_mask, reference_transform, reference_crs, reference_shape):
    """
    Calculer ETb pour une année donnée
    
    Parameters:
    -----------
    year : int
        Année à traiter
    cropland_mask : np.ndarray
        Masque des terres cultivées
    reference_transform : affine.Affine
        Transformation de référence
    reference_crs : CRS
        Système de coordonnées de référence
    reference_shape : tuple
        Dimensions (height, width) de référence
    
    Returns:
    --------
    etb_annual : np.ndarray
        ETb annuel (mm/an)
    etb_cropland : np.ndarray
        ETb masqué sur les terres cultivées
    stats : dict
        Statistiques de ETb
    """
    
    print(f"\n{'='*60}")
    print(f"  Année {year}")
    print(f"{'='*60}")
    
    # Charger AETI
    et_file = f"{data_dir}/ET/AETI_L1_{year}.tif"
    with rasterio.open(et_file) as src:
        aeti_annual = src.read(1).astype(np.float32)
        aeti_annual[aeti_annual < 0] = np.nan
        aeti_transform = src.transform
        aeti_crs = src.crs
    
    print(f"✓ AETI chargé: {aeti_annual.shape}")
    
    # Charger PCP
    pcp_file = f"{data_dir}/PCP/PCP_L1_{year}.tif"
    with rasterio.open(pcp_file) as src:
        pcp_raw = src.read(1).astype(np.float32)
        pcp_raw[pcp_raw < 0] = np.nan
        pcp_transform = src.transform
        pcp_crs = src.crs
    
    print(f"✓ PCP chargé: {pcp_raw.shape}")
    
    # Aligner PCP si nécessaire
    if pcp_raw.shape != reference_shape:
        print(f"  ⚠️ Rééchantillonnage PCP: {pcp_raw.shape} → {reference_shape}")
        pcp_annual = align_raster_to_reference(
            pcp_raw, pcp_transform, pcp_crs,
            reference_shape, reference_transform, reference_crs,
            resampling_method=Resampling.bilinear
        )
    else:
        pcp_annual = pcp_raw
    
    # Aligner AETI si nécessaire
    if aeti_annual.shape != reference_shape:
        print(f"  ⚠️ Rééchantillonnage AETI: {aeti_annual.shape} → {reference_shape}")
        aeti_aligned = align_raster_to_reference(
            aeti_annual, aeti_transform, aeti_crs,
            reference_shape, reference_transform, reference_crs,
            resampling_method=Resampling.bilinear
        )
        aeti_annual = aeti_aligned
    
    # Calculer P_effective (approximation)
    pcp_effective = 0.7 * pcp_annual
    
    # Calculer ETb
    etb_annual = np.maximum(aeti_annual - pcp_effective, 0)
    
    # Appliquer le masque cropland
    etb_cropland = etb_annual * cropland_mask
    etb_cropland[cropland_mask == 0] = np.nan
    
    # Statistiques
    stats = {
        'year': year,
        'etb_min': float(np.nanmin(etb_cropland)),
        'etb_max': float(np.nanmax(etb_cropland)),
        'etb_mean': float(np.nanmean(etb_cropland)),
        'etb_median': float(np.nanmedian(etb_cropland)),
        'etb_std': float(np.nanstd(etb_cropland)),
        'aeti_mean': float(np.nanmean(aeti_annual)),
        'pcp_mean': float(np.nanmean(pcp_annual))
    }
    
    print(f"\n📊 Statistiques ETb {year} (cropland):")
    print(f"  Moyenne: {stats['etb_mean']:.1f} mm/an")
    print(f"  Médiane: {stats['etb_median']:.1f} mm/an")
    print(f"  Min:     {stats['etb_min']:.1f} mm/an")
    print(f"  Max:     {stats['etb_max']:.1f} mm/an")
    print(f"  Écart-type: {stats['etb_std']:.1f} mm/an")
    
    return etb_annual, etb_cropland, stats

print("✓ Fonction calculate_etb définie")

## 3. Fonction de Calcul ETb

Calculer l'évapotranspiration bleue (eau d'irrigation)

In [ ]:
def align_raster_to_reference(source_data, source_transform, source_crs, 
                              reference_shape, reference_transform, reference_crs,
                              resampling_method=Resampling.bilinear):
    """
    Aligner un raster source à une grille de référence
    
    Parameters:
    -----------
    source_data : np.ndarray
        Données du raster source
    source_transform : affine.Affine
        Transformation du raster source
    source_crs : CRS
        Système de coordonnées du source
    reference_shape : tuple
        Dimensions (height, width) de référence
    reference_transform : affine.Affine
        Transformation de référence
    reference_crs : CRS
        Système de coordonnées de référence
    resampling_method : Resampling
        Méthode de rééchantillonnage (bilinear pour variables continues)
    
    Returns:
    --------
    aligned_data : np.ndarray
        Raster aligné sur la grille de référence
    """
    # Créer tableau de destination
    aligned_data = np.zeros(reference_shape, dtype=np.float32)
    
    # Reprojeter
    reproject(
        source=source_data,
        destination=aligned_data,
        src_transform=source_transform,
        src_crs=source_crs,
        dst_transform=reference_transform,
        dst_crs=reference_crs,
        resampling=resampling_method
    )
    
    return aligned_data

print("✓ Fonction d'alignement définie")

## 2. Fonction d'Alignement Spatial

Fonction pour aligner automatiquement les rasters à la grille de référence

In [ ]:
# Chemins
data_dir = "../data/raw"
processed_dir = "../data/processed"
mask_file = f"{processed_dir}/cropland_mask_wapor.tif"

# Charger le masque cropland
print("🌾 Chargement du masque cropland...\n")

with rasterio.open(mask_file) as src:
    cropland_mask = src.read(1)
    mask_profile = src.profile
    mask_transform = src.transform
    mask_crs = src.crs

print(f"✓ Masque chargé: {mask_file}")
print(f"  Dimensions: {cropland_mask.shape}")
print(f"  Pixels cropland > 0: {np.sum(cropland_mask > 0):,}")
print(f"  Pixels cropland > 50%: {np.sum(cropland_mask > 0.5):,}")
print(f"  Surface cropland (estimée): ~{np.sum(cropland_mask) * 0.09:.0f} km²")

# Visualisation rapide
plt.figure(figsize=(10, 6))
plt.imshow(cropland_mask, cmap='Greens', interpolation='nearest')
plt.colorbar(label='Fraction cropland [0-1]')
plt.title('Masque des Terres Cultivées - Tunisie', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 1. Charger le Masque Cropland

Utiliser le masque créé dans le notebook 02

In [ ]:
# Imports
import sys
sys.path.append('..')

import os
import numpy as np
import pandas as pd
import rasterio
from rasterio.warp import reproject, Resampling
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
from datetime import datetime

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Modules importés")
print(f"📅 Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")